In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import os 


In [2]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
URL = 'https://www.ankhang.vn/laptop-theo-hang.html'
driver.get(URL)

In [3]:
# get brand__laptop_links

link_laptop_brands = []

container_brands = driver.find_element(By.CSS_SELECTOR, '.filter-holder.bg-white.h-100 .filter-left')
lable_brands = container_brands.find_elements(By.CSS_SELECTOR, 'label')
print(f"Số hãng laptop tìm thấy: {len(lable_brands)}")
print(type(lable_brands[0]))

for lable in lable_brands: 
    print(f"lable_text : {lable.text}")
    link_str = lable.get_attribute('onclick')
    if link_str:
        print(link_str)
        start = link_str.find("'") + 1
        end = link_str.rfind("'")
        link = link_str[start:end]
        link_laptop_brands.append(link)
print(f"totals link laptop brands: {len(link_laptop_brands)}")


Số hãng laptop tìm thấy: 10
<class 'selenium.webdriver.remote.webelement.WebElement'>
lable_text : Laptop Acer (147)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=135'
lable_text : Laptop Asus (375)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=134'
lable_text : Laptop Dell (149)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=137'
lable_text : Laptop HP (515)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=128'
lable_text : Laptop Lenovo (599)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=165'
lable_text : Laptop MSI (122)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=199'
lable_text : Apple (2)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=329'
lable_text : Gigabyte (40)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=59'
lable_text : Laptop LG (9)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=2'
lable_text : Sam

In [ ]:
# for idx, link in enumerate(link_laptop_brands):
#     print(f"in link: {link}")
#     try:
#         driver.get(link)

#         #wait for page loading
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.TAG_NAME, 'body'))
#         )
#         print(f"loaded link: {idx} scuccessfully")
        



#     except Exception as e:
#         print(f"error when crawl link: {idx}, error:{e}")
        


in link: https://www.ankhang.vn/laptop-theo-hang.html?brand=135
loaded link: 0 scuccessfully
in link: https://www.ankhang.vn/laptop-theo-hang.html?brand=134
loaded link: 1 scuccessfully
in link: https://www.ankhang.vn/laptop-theo-hang.html?brand=137
loaded link: 2 scuccessfully
in link: https://www.ankhang.vn/laptop-theo-hang.html?brand=128
loaded link: 3 scuccessfully
in link: https://www.ankhang.vn/laptop-theo-hang.html?brand=165
loaded link: 4 scuccessfully
in link: https://www.ankhang.vn/laptop-theo-hang.html?brand=199
loaded link: 5 scuccessfully
in link: https://www.ankhang.vn/laptop-theo-hang.html?brand=329
loaded link: 6 scuccessfully
in link: https://www.ankhang.vn/laptop-theo-hang.html?brand=59
loaded link: 7 scuccessfully
in link: https://www.ankhang.vn/laptop-theo-hang.html?brand=2
loaded link: 8 scuccessfully
in link: https://www.ankhang.vn/laptop-theo-hang.html?brand=349
loaded link: 9 scuccessfully


In [19]:
def crawl_href_item():
    link_items = []

    item_containers = driver.find_elements(By.CSS_SELECTOR, '.product_list_box .p-item-2021 .p-container')
    for item in item_containers:
        try:
            link_item = item.find_element(By.TAG_NAME, 'a').get_attribute('href')
            link_items.append(link_item)
            
        except Exception as e:
            print(f"loi khi lay href item: {e}")
    return link_items
    

def crawl_detail_item():
    pass

def go_to_next_page():
    try:
        # lay block phan trang
        paging = driver.find_element(By.CSS_SELECTOR, '.paging')

        #lay trang hien tai
        curent_page_ele = paging.find_element(By.CSS_SELECTOR, '.current')
        current_page = int(curent_page_ele.text.strip())

        # lay danh sach trang
        page_nums = []
        page_num_elements =  paging.find_elements(By.TAG_NAME, 'a')
        for ele in page_num_elements:
            try:
                if ele.text.strip().isdigit():
                    page_nums.append(
                        {
                            'page_num' : int(ele.text.strip()),
                            'href' : ele.get_attribute('href')
                        }
                    )
            except Exception as e:
                print(f"loi khi lay href paging : {e}")
        
        # tim trang tiep theo
        next_page_num = None
        next_page_href = None

        for page_ele in page_nums:
            if page_ele['page_num'] > current_page:
                next_page_num = page_ele['page_num']
                next_page_href = page_ele['href']
                break
        
        return next_page_num, next_page_href
    except Exception as e:
        print(f"loi khi lay trang tiep theo: {e}")
        return None, None



In [ ]:
driver.get(link_laptop_brands[0])
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, 'body'))
)
print(f"loaded link scuccessfully")
while True:
    try:
        # crawl href item
        link_items = crawl_href_item()
        if link_items:
            print(f"tim thay {len(link_items)} link item")
            crawl_detail_item()
            
            # crawl chi tiet item
            for link in link_items:
                driver.get(link)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, 'body'))
                )
                crawl_detail_item()
                

        
        # chuyen trang tiep theo
        next_page_num, next_page_href = go_to_next_page()
        if next_page_href:
            print(f"dang chuyen den trang {next_page_num} - {next_page_href}")
            driver.get(next_page_href)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, 'body'))
            )
        else:
            print(f"da het trang de crawl")
            break
    except Exception as e:
        print(f"loi khi crawl trang: {e}")
        

        


loaded link scuccessfully
tim thay 20 link item
dang chuyen den trang 2 - https://www.ankhang.vn/laptop-theo-hang.html?brand=135&page=2
tim thay 20 link item
dang chuyen den trang 3 - https://www.ankhang.vn/laptop-theo-hang.html?brand=135&page=3
tim thay 20 link item
dang chuyen den trang 4 - https://www.ankhang.vn/laptop-theo-hang.html?brand=135&page=4
tim thay 20 link item
dang chuyen den trang 5 - https://www.ankhang.vn/laptop-theo-hang.html?brand=135&page=5
tim thay 20 link item
dang chuyen den trang 6 - https://www.ankhang.vn/laptop-theo-hang.html?brand=135&page=6
tim thay 20 link item
dang chuyen den trang 7 - https://www.ankhang.vn/laptop-theo-hang.html?brand=135&page=7
tim thay 20 link item
dang chuyen den trang 8 - https://www.ankhang.vn/laptop-theo-hang.html?brand=135&page=8
tim thay 7 link item
da het trang de crawl
